In [1]:
%run "../../shared/utilz_includez.ipynb"

In [2]:
from skimage import io, img_as_float, img_as_ubyte
from skimage import color , measure

from scipy import ndimage as nd

import cv2 

from skimage.segmentation import clear_border

## Non-local-means denoising Approach @microscope if not using cv2.median
from skimage.restoration import denoise_nl_means, estimate_sigma

In [3]:
## setup fake webcam
# !sudo modprobe v4l2loopback
#### --- desktop record grab_x, grab_y
# !ffmpeg -f x11grab -r 15 -s 960x720 -i :1 -vcodec rawvideo -pix_fmt yuv420p -threads 0 -f v4l2 /dev/video0
#### --- file stream 
# !export VIDE0=~/Videos/webcam_sim.mp4
# !ffmpeg -re -stream_loop -1 -i $VIDEO -map 0:v -f mpegts udp://localhost:50000
  

In [4]:
def plot_image( img_array , plotit=True, title=None, cmapd='gray', logit=True):     
    if logit:
        print( f"\n{ '-'*7 } { title if title else type(img_array) } { '-'*7 }" )
        print( f"image.shape = {img_array.shape}" ) 
        print( f"datatype = {img_array.dtype}")
        print( f"min = {np.min(img_array)} , max = {np.max(img_array)}\n" )
    if plotit:
        if cmapd:
            plt.imshow( img_array , cmap=cmapd)
        else:
            plt.imshow( img_array )
    if plotit and title:
        plt.title(title)
        
        
        
def fetch_image(imgpath, forceuint8=True):
    img = io.imread( imgpath )
    if forceuint8:
        img = np.uint8(img)
    if len(img.shape) > 2: ##hack gray check << TODO: channel 4 ???
        gimg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY ) 
    else:
        gimg = img.copy()
    return img, gimg
    
def grid_plot_images( imgz, titlz=None, plotit=True, logit=True, cmapd='gray', nr=1, nc=2):
    nr = np.int( len(imgz) / nc )
    if len(imgz) % nc != 0:
        nr = nr+1
    for i in range( len(imgz) ):
        plt.subplot(nr, nc, i+1 )
        plot_image( imgz[i], title=titlz[i] if titlz else None , plotit=plotit, logit=logit, cmapd=cmapd); 
        

In [8]:
## fetch list of imagez and return as dict
def fetch_imagez_dict( imgpath_list, lbl_prefix="Image", logit=False):
    dict_cell_imagez = {} 
    for i, ip in enumerate(imgpath_list):
        c, g = fetch_image( ip )
        if logit:
            print(f"IP at {i} is okay")
        dict_cell_imagez[f"{lbl_prefix} # {i}"] = (c, g )
        
    return dict_cell_imagez

In [5]:
## -- openCV stuff

def plot_cv2Image(img, title='Image View'):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

def plot_cv2Video(capture, title='Video View'):
    while capture.isOpened(): ## go frame by frame
        success, img = capture.read()
        if success:
            cv2.imshow( title, img)
        if cv2.waitKey(30) & 0xFF == 27: ##ESC  # 1 == ord('q'):
            break
      
    capture.release()
    cv2.destroyAllWindows()  
    
    

def cv2_video_capture(src_id, img_handler=None, title='Generic', width=640, height=480, intensity=150, logit=False):    
    if logit:
        print(f"@cv2_video_capture: Starting '{title}' with img_handler as {img_handler}" ) 
            
    cap = cv2.VideoCapture( src_id ) 
    cap.set(3, width) ##width is id 3
    cap.set(4, height) ##height is id 4
    cap.set(10, intensity)
    
    while cap.isOpened(): ## go frame by frame
        success, frame = cap.read()
        if success:
            if img_handler: 
                frame = img_handler( frame , logit)
            else:
                _ = cv2.putText(frame, 
                                f'NOOP@img_handler={img_handler}',
                               (20,40), 
                                cv2.FONT_HERSHEY_COMPLEX, 
                                1, (0,0,255), 2) 
            cv2.imshow( title, frame)
        if cv2.waitKey(30) & 0xFF == 27: ##ESC  # 1 == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

    
## Seek contour and bounding box of the object
def get_contour_box(mask_img, out_img, athresh=500, tfact=0.02):
    contz, hier = cv2.findContours(mask_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE ) 
    appx_list = []
    for c in contz:
        a = cv2.contourArea( c )
        if a > athresh:
            cv2.drawContours( out_img, c, -1, (200,0,255), 3)
            peri = cv2.arcLength(c, True )
            appx = cv2.approxPolyDP(c, tfact*peri, True)
            appx_list.append( appx )
    ## return tip of object
    return appx_list
    

## Preprocess for edges
def cv2_preprocess_edges(img, kern=(5,5), blursig=1, dil_iterz=1, canny_thresh1=200, canny_thresh2=200):    
    gimg = img.copy()
    
    # b. grayscale it for filterz
    if len( img.shape) > 2: ##hack check if grayscale
        gimg = cv2.cvtColor( img, cv2.COLOR_BGR2GRAY )
       
    
    # c. blur and edge detect
    img_edgez = cv2.Canny( cv2.GaussianBlur(gimg, kern, blursig), canny_thresh1, canny_thresh2)
    img_edgez = cv2.erode( 
            cv2.dilate(img_edgez, np.ones(kern), iterations=dil_iterz), 
            np.ones(kern), iterations=1)
    
    return img_edgez

In [6]:
## Warp image to get bird's eye view
def reshape_box_approx(bbox):
    tmp_ptz = bbox.reshape( (4,2) )
    
    ptz = np.zeros( (4,1,2), np.int32 )
    
    ## use sum to identify origins as the smallest and (x+w,y+h) as the largest
    sumz = tmp_ptz.sum(axis=1)
    #print( f'Sumz = {sumz}')
    ## use diff to identify w and h size
    diffz = np.diff(tmp_ptz, axis=1)
    #print( f'Diffz = {diffz}')
        
    ptz[0] = tmp_ptz[ np.argmin(sumz) ] ## origin
    ptz[3] = tmp_ptz[ np.argmax(sumz) ] ## endpoint (origin + (w,h))
    ptz[1] = tmp_ptz[ np.argmin(diffz) ] ## w
    ptz[2] = tmp_ptz[ np.argmax(diffz) ] ## h     
    #print( f"MAP:\n{ptz}\n<<<<<<< END")
    
    return ptz

def get_warp_perspective(img, biggest_approx, imgW=480, imgH=640):     
    #print( ">>>> ", biggest_approx , "\n")    
    pts1 = np.float32( reshape_box_approx(biggest_approx) )
    pts2 = np.float32([[0,0], [imgW,0], [0, imgH], [imgW, imgH]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    warped_img = cv2.warpPerspective(img, matrix, (imgW, imgH) )    
               
    # e. crop edges to clean up
    w, h, *_ = warped_img.shape 
    warped_img = warped_img[ edge_margin:w-edge_margin, edge_margin:h-edge_margin]
    # f.resize to fit screen
    _ = cv2.resize(warped_img, (imgW, imgH) )
    
    return warped_img

In [7]:
blue_channel = 2

def nuclei_channel_only(img):
    return img[:, :, blue_channel]

In [9]:
## gaussian kernel creator

In [10]:
## clean, segment using watershed and generate properties
kern = np.ones( (3,3), dtype=np.uint8 )

pixels_to_um = 0.5 

radianz = 57.2958


region_propz = ['Label', "Area", "Perimeter", 
     "equivalent_diameter", "orientation",
    "MajorAxisLength", "MinorAxisLength",
     "MinIntensity", "MaxIntensity", "MeanIntensity",
    ]
propz_addz = ["Area_nm", "Perimeter_nm", 
     "equivalent_diameter_nm", 
    "orientation_degrees",
    "MajorAxisLength_nm", "MinorAxisLength_nm",]


def nonlocalmeans_clean(gimg, hfactz=1.15, patchz=5, patch_distz=3, fastz=False):
    sig = np.mean( estimate_sigma(gimg, multichannel=True) ) 
    oimg = denoise_nl_means(gimg, h=hfactz*sig, fast_mode=fastz, patch_size=patchz, patch_distance=patch_distz, multichannel=True)
    return oimg #np.astype(oimg, np.uint8)

def clean_prepare_binary( gimg , blursize=3):
    # a. binary threshold 
    otsu, timg = cv2.threshold(gimg, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    # b. erode dilate @ crispier grains
    timg = cv2.erode( timg, kern, iterations=1)
    timg = cv2.dilate( timg, kern, iterations=1)

    # c. median blur pixeletation << should this go first 
    timg = cv2.medianBlur(timg, blursize)
    
    return timg

def watershed_segment(cimg, gimg, thresh_val=0.2, kern=np.ones((3,3), dtype=np.uint8), iterz=1, dilate_iterz=1 ): 
    # a. morphological extra 
    mimg = cv2.morphologyEx( gimg, cv2.MORPH_OPEN, kern, iterations=iterz)
    
    # b. clear border cells/grains at this point 
    mimg = clear_border( mimg )
    
    # c. Identify sure foreground pixels     
    sure_bg = cv2.dilate( mimg, kern, iterations=dilate_iterz) 
    mimg = cv2.distanceTransform( mimg, cv2.DIST_L2, 3)
    
    # d.  Threshold
    _, sure_fg = cv2.threshold(mimg, thresh_val*mimg.max(),  255, 0)
    sure_fg = np.uint8( sure_fg )
    
    # e. gen markers and distinguish unkownz from background 
    _, markerz = cv2.connectedComponents( sure_fg )
    
    unknownz = cv2.subtract( sure_bg, sure_fg)
    markerz = markerz+10
    markerz[ unknownz == 255 ] = 0 
    
    # f. watershed
    watershed = cv2.watershed( cimg, markerz )

    ready_img = cimg.copy()
    ready_img[ markerz == -1] = [0, 255, 255]
    rimg2 = color.label2rgb( markerz, bg_label=0 )
    
    ## return markerz, rgb labeled images
    return markerz, rimg2

def gen_properties(markerz, gimg): 
    clusterz = measure.regionprops(markerz, intensity_image=gimg)
    g = clusterz[0]
    
    ## Create dframe for it
    dd = []
    for g in clusterz:
        gl = []
        for p in region_propz:
            gl.append( g[p] )

        for ap in propz_addz:
            p = "_".join( ap.split("_")[:-1])

            ## Area_nm
            if( p == 'Area'):
                gl.append( g[p] * pixels_to_um**2 )
            ## Orientation_nm
            elif( p == 'orientation'):
                gl.append( g[p]* radianz ) 
            ## anything that's not Intensity
            else:
                gl.append( g[p]* pixels_to_um ) 


    dd.append( gl )
    
    # df = pd.DataFrame.from_records( dd , columns=propz+addz )
    ## return list oobject 
    return dd



    

NameError: name 'np' is not defined

In [ ]:
## Descriptors and Matching


In [ ]:
## Fourier Transform
def run_discrete_fourier_transform(gimg):    
    ## a. generate dft 
    dft = cv2.dft( np.float32(gimg), flags=cv2.DFT_COMPLEX_OUTPUT )

    ## b. by default data is centered around the origin and the origin is at top left corner. So move the origin to the center of the grid
    # shift the zero frequency to center
    dft = np.fft.fftshift( dft)

    ## c. compute the magnitude spectrum 
    kil_zero = 0
    dft_ms = 20 * np.log( (cv2.magnitude(dft[:,:,0], dft[:,:,1]) )+kil_zero )# +1 to avoid div by zero error
    
    return dft, dft_ms